In [3]:
#load_ext autotime
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm.notebook import tqdm_notebook
import os
import numpy as np
import glob

In [2]:
locator = Nominatim(user_agent="measurements")

In [3]:
inpath = r"T:\DCProjects\StoryMap\BikeCounting\BikeShare\Data\lastTrips"

In [4]:
odf = pd.read_csv(os.path.join(inpath, "trips_peace_health_rides_05_01_2019-05_31_2019.csv"))
odf.head()

,User ID,Route ID,Start Hub,Start Area,Start Latitude,Start Longitude,Start Date,Start Week Day,Start Time,End Hub,...,Bike ID,Bike Name,Distance [Miles],Duration,Multiple Rental,Ride Cost,Fees,Bonuses,Total Cost,Payment plan
0,762507,11291076,NaN,Phase 1 System Area,44.045575,-123.0945317,5/1/2019,Wednesday,0:04,13th & Lawrence,...,19793,263,0.21,0:03:07,False,1.0,0,-1,0.0,Single Ride
1,775345,11291083,17th & Hilyard,Phase 1 System Area,44.04107,-123.0819067,5/1/2019,Wednesday,0:10,"17th & Pearl, East side",...,19717,187,0.38,0:03:37,False,0.0,0,0,0.0,UO Single Ride
2,784725,11291122,19th and Agate,Phase 1 System Area,44.03919,-123.0695917,5/1/2019,Wednesday,1:03,12th & Willamette,...,19823,293,1.76,0:09:34,False,0.0,0,0,0.0,UO Monthly Plan
3,762507,11291148,13th & Lawrence,Phase 1 System Area,44.045325,-123.0984033,5/1/2019,Wednesday,1:37,Broadway & Willamette,...,19793,263,0.59,0:07:48,False,0.0,0,0,0.0,Monthly Plan
4,762507,11291173,10th & Olive SE,Phase 1 System Area,44.04917667,-123.0943933,5/1/2019,Wednesday,2:27,13th & Lawrence,...,19700,170,0.48,0:05:52,False,0.0,0,0,0.0,Monthly Plan


In [7]:
odf.columns

Index(['User ID', 'Route ID', 'Start Hub', 'Start Area', 'Start Latitude',
       'Start Longitude', 'Start Date', 'Start Week Day', 'Start Time',
       'End Hub', 'End Area', 'End Latitude', 'End Longitude', 'End Date',
       'End Time', 'Bike ID', 'Bike Name', 'Distance [Miles]', 'Duration',
       'Multiple Rental', 'Ride Cost', 'Fees', 'Bonuses', 'Total Cost',
       'Payment plan'],
      dtype='object')

In [5]:
df = odf 

In [12]:
df = df[(df['Start Latitude'] != " - ") | (df['End Latitude'] != " - " ) | (df['Start Longitude'] != " - ") | (df['End Longitude'] != " - ")]

In [13]:
df["startgeom"] = df["Start Latitude"].map(str) + ',' + df['Start Longitude'].map(str)
df["startgeom"][0]

'44.045575,-123.0945317'

In [14]:
df["endgeom"] = df["End Latitude"].map(str) + ',' + df['End Longitude'].map(str)
df["endgeom"][0]

'44.04532667,-123.098445'

In [15]:
locator = Nominatim(user_agent="measurements", timeout=10)
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.00001)

In [16]:
tqdm_notebook.pandas()

In [17]:
df['startaddress'] = df['startgeom'].progress_apply(rgeocode)
df.head()

  0%|          | 0/27667 [00:00<?, ?it/s]

,User ID,Route ID,Start Hub,Start Area,Start Latitude,Start Longitude,Start Date,Start Week Day,Start Time,End Hub,...,Duration,Multiple Rental,Ride Cost,Fees,Bonuses,Total Cost,Payment plan,startgeom,endgeom,startaddress
0,762507,11291076,NaN,Phase 1 System Area,44.045575,-123.0945317,5/1/2019,Wednesday,0:04,13th & Lawrence,...,0:03:07,False,1.0,0,-1,0.0,Single Ride,"44.045575,-123.0945317","44.04532667,-123.098445","(13th & Olive, Olive Alley, Eugene, Lane Count..."
1,775345,11291083,17th & Hilyard,Phase 1 System Area,44.04107,-123.0819067,5/1/2019,Wednesday,0:10,"17th & Pearl, East side",...,0:03:37,False,0.0,0,0,0.0,UO Single Ride,"44.04107,-123.0819067","44.04118333,-123.0894917","(Peace Lutheran, East 17th Avenue, College Hil..."
2,784725,11291122,19th and Agate,Phase 1 System Area,44.03919,-123.0695917,5/1/2019,Wednesday,1:03,12th & Willamette,...,0:09:34,False,0.0,0,0,0.0,UO Monthly Plan,"44.03919,-123.0695917","44.04650333,-123.0929017","(University of Oregon, Sunset Alley, Eugene, L..."
3,762507,11291148,13th & Lawrence,Phase 1 System Area,44.045325,-123.0984033,5/1/2019,Wednesday,1:37,Broadway & Willamette,...,0:07:48,False,0.0,0,0,0.0,Monthly Plan,"44.045325,-123.0984033","44.04964836,-123.0926925","(1307, Lawrence Street, Eugene, Lane County, O..."
4,762507,11291173,10th & Olive SE,Phase 1 System Area,44.04917667,-123.0943933,5/1/2019,Wednesday,2:27,13th & Lawrence,...,0:05:52,False,0.0,0,0,0.0,Monthly Plan,"44.04917667,-123.0943933","44.04529333,-123.0984417","(Lane Community College Downtown Campus, West ..."


In [18]:
df['endaddress'] = df['endgeom'].progress_apply(rgeocode)
df.head()

  0%|          | 0/27667 [00:00<?, ?it/s]

,User ID,Route ID,Start Hub,Start Area,Start Latitude,Start Longitude,Start Date,Start Week Day,Start Time,End Hub,...,Multiple Rental,Ride Cost,Fees,Bonuses,Total Cost,Payment plan,startgeom,endgeom,startaddress,endaddress
0,762507,11291076,NaN,Phase 1 System Area,44.045575,-123.0945317,5/1/2019,Wednesday,0:04,13th & Lawrence,...,False,1.0,0,-1,0.0,Single Ride,"44.045575,-123.0945317","44.04532667,-123.098445","(13th & Olive, Olive Alley, Eugene, Lane Count...","(1306, Lawrence Street, Eugene, Lane County, O..."
1,775345,11291083,17th & Hilyard,Phase 1 System Area,44.04107,-123.0819067,5/1/2019,Wednesday,0:10,"17th & Pearl, East side",...,False,0.0,0,0,0.0,UO Single Ride,"44.04107,-123.0819067","44.04118333,-123.0894917","(Peace Lutheran, East 17th Avenue, College Hil...","(The Pearl, 1661, Pearl Street, College Hill, ..."
2,784725,11291122,19th and Agate,Phase 1 System Area,44.03919,-123.0695917,5/1/2019,Wednesday,1:03,12th & Willamette,...,False,0.0,0,0,0.0,UO Monthly Plan,"44.03919,-123.0695917","44.04650333,-123.0929017","(University of Oregon, Sunset Alley, Eugene, L...","(13th and Olive (Willamette), West 13th Avenue..."
3,762507,11291148,13th & Lawrence,Phase 1 System Area,44.045325,-123.0984033,5/1/2019,Wednesday,1:37,Broadway & Willamette,...,False,0.0,0,0,0.0,Monthly Plan,"44.045325,-123.0984033","44.04964836,-123.0926925","(1307, Lawrence Street, Eugene, Lane County, O...","(Kesey Square, Eugene, Lane County, Oregon, Un..."
4,762507,11291173,10th & Olive SE,Phase 1 System Area,44.04917667,-123.0943933,5/1/2019,Wednesday,2:27,13th & Lawrence,...,False,0.0,0,0,0.0,Monthly Plan,"44.04917667,-123.0943933","44.04529333,-123.0984417","(Lane Community College Downtown Campus, West ...","(1308, Lawrence Street, Eugene, Lane County, O..."


In [22]:
def getLoc(x):
    if x is not None:
        loc = x.raw['address']
        a = [key for key, value in loc.items()][0]
        b = [key for key, value in loc.items()][1]
        if  a == 'house_number':
            res = loc['road'] + ' ' + loc['house_number']
        else:
            res = loc[a]
            if res == "University of Oregon":
                res = "UO " + loc[b]
    else:
        res = ""
    return res

In [29]:
df['startname'] = df.startaddress.apply(lambda x: getLoc(x))

In [30]:
df['endname'] = df.endaddress.apply(lambda x: getLoc(x))

In [32]:
df.columns

Index(['User ID', 'Route ID', 'Start Hub', 'Start Area', 'Start Latitude',
       'Start Longitude', 'Start Date', 'Start Week Day', 'Start Time',
       'End Hub', 'End Area', 'End Latitude', 'End Longitude', 'End Date',
       'End Time', 'Bike ID', 'Bike Name', 'Distance [Miles]', 'Duration',
       'Multiple Rental', 'Ride Cost', 'Fees', 'Bonuses', 'Total Cost',
       'Payment plan', 'startgeom', 'endgeom', 'startaddress', 'endaddress',
       'startname', 'endname'],
      dtype='object')

In [44]:
df.loc[df['Start Hub'].isnull(), 'Start Hub'] = df.loc[df['Start Hub'].isnull(), 'startname']

In [45]:
df.loc[df['End Hub'].isnull(), 'End Hub'] = df.loc[df['End Hub'].isnull(), 'endname']

In [50]:
df.to_csv(os.path.join(inpath, "trips_peace_health_rides_05_01_2019-05_31_2019_updated.csv"), index=False)

In [51]:
ndf = df.drop(['startgeom', 'endgeom', 'startaddress', 'endaddress',
       'startname', 'endname'], axis=1)

In [54]:
outpath = r"T:\DCProjects\StoryMap\BikeCounting\BikeShare\Data\Output"

In [55]:
ndf.to_csv(os.path.join(outpath, "trips_peace_health_rides_05_01_2019-05_31_2019.csv"), index=False)

In [1]:
path = r"T:\DCProjects\StoryMap\BikeCounting\BikeShare\Data\Output\review"

In [5]:
files = glob.glob(os.path.join(path, "*.csv"))

In [50]:
def review_hubs(re_df, strp='Eugene'):
    for hub in ["Start", "End"]:
        if hub+' Hub' in re_df.columns:
            re = re_df[re_df[hub+' Hub'].str.contains(strp)][[hub+' Latitude', hub+' Longitude']]
        else:
            re = re_df[re_df[hub+'.Hub'].str.contains(strp)][[hub+'.Latitude', hub+'.Longitude']]
        
        if re.shape[0] > 0:
            re.columns = ['Lat', 'Lon']
            re["geom"] = re["Lat"].map(str) + ',' + re['Lon'].map(str)
            locator = Nominatim(user_agent="measurements", timeout=10)
            rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.00001)
            tqdm_notebook.pandas()
            re['address'] = re['geom'].progress_apply(rgeocode)
            re['name'] =re.address.apply(getLoc)
            if hub+' Hub' in re_df.columns:
                re_df.loc[re_df[hub+' Hub'].str.contains(strp), hub+' Hub'] = re.name.values  
            else:
                re_df.loc[re_df[hub+'.Hub'].str.contains(strp), hub+'.Hub'] = re.name.values
    return re_df

In [20]:
def review_hubs(re_df):
    for hub in ["Start", "End"]:
        if hub+' Hub' in re_df.columns:
            if re_df[re_df[hub+' Hub'].isnull()].shape[0] > 0:
                re = re_df[re_df[hub+' Hub'].isnull()][[hub+' Latitude', hub+' Longitude']]
            else:
                re = re_df[re_df[hub+' Hub'].str.isnumeric()][[hub+' Latitude', hub+' Longitude']]
        else:
            re = re_df[re_df[hub+'.Hub'].str.isnumeric()][[hub+'.Latitude', hub+'.Longitude']]
        
        if re.shape[0] > 0:
            re.columns = ['Lat', 'Lon']
            re["geom"] = re["Lat"].map(str) + ',' + re['Lon'].map(str)
            locator = Nominatim(user_agent="measurements", timeout=10)
            rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.00001)
            tqdm_notebook.pandas()
            re['address'] = re['geom'].progress_apply(rgeocode)
            re['name'] =re.address.apply(getLoc)
            if hub+' Hub' in re_df.columns:
                if re_df[re_df[hub+' Hub'].isnull()].shape[0] > 0:
                    re_df.loc[re_df[hub+' Hub'].isnull(), hub+' Hub'] = re.name.values
                else:
                    re_df.loc[re_df[hub+' Hub'].str.isnumeric(), hub+' Hub'] = re.name.values  
            else:
                re_df.loc[re_df[hub+'.Hub'].str.isnumeric(), hub+'.Hub'] = re.name.values
    return re_df

In [52]:
for file in files:
    print(file)
    re_df = pd.read_csv(file)
    #re_df = review_hubs(re_df)
    re_df = review_hubs(re_df, strp='University of')
    re_df.to_csv(file, index=False)

T:\DCProjects\StoryMap\BikeCounting\BikeShare\Data\Output\review\trips_peace_health_rides_01_01_2019-01_31_2019.csv


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

T:\DCProjects\StoryMap\BikeCounting\BikeShare\Data\Output\review\trips_peace_health_rides_01_01_2020-01_31_2020.csv


  0%|          | 0/46 [00:00<?, ?it/s]

...

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

T:\DCProjects\StoryMap\BikeCounting\BikeShare\Data\Output\review\trips_peace_health_rides_12_01_2019-12_31_2019.csv


  0%|          | 0/63 [00:00<?, ?it/s]

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [39]:
file = r"T:\DCProjects\StoryMap\BikeCounting\BikeShare\Data\Output\review\trips_peace_health_rides_12_01_2019-12_31_2019.csv"
file

'T:\\DCProjects\\StoryMap\\BikeCounting\\BikeShare\\Data\\Output\\review\\trips_peace_health_rides_12_01_2019-12_31_2019.csv'

In [40]:
re_df = pd.read_csv(file)

In [41]:
re_df.columns

Index(['User ID', 'Route ID', 'Bike Region ID', 'Payment Plan', 'Start Hub',
       'Start Area', 'Start Special Area', 'Start Latitude', 'Start Longitude',
       'Start Date', 'Start Time', 'End Hub', 'End Area', 'End Special Area',
       'End Latitude', 'End Longitude', 'End Date', 'End Time', 'Member Type',
       'Trip Type', 'Bike ID', 'Bike Name', 'Distance [Miles]', 'Duration',
       'Rental Access Path', 'Multiple Rental', 'Ride cost', 'Fees', 'Bonuses',
       'Total cost'],
      dtype='object')

In [42]:
re_df.head()

,User ID,Route ID,Bike Region ID,Payment Plan,Start Hub,Start Area,Start Special Area,Start Latitude,Start Longitude,Start Date,...,Bike ID,Bike Name,Distance [Miles],Duration,Rental Access Path,Multiple Rental,Ride cost,Fees,Bonuses,Total cost
0,766963,12817400,NaN,UO Monthly Plan,Uniquely Chengdu,Phase 1 System Area,NaN,44.045435,-123.080060,2019-12-01,...,19591,0057,0.76,00:07:59,keypad,False,0.0,0.0,0.0,0.0
1,1702290,12817409,NaN,UO Monthly Plan,Thompson-Roach Building,Phase 1 System Area,NaN,44.045438,-123.084452,2019-12-01,...,19703,0173,0.35,00:13:51,keypad,True,0.0,0.0,0.0,0.0
2,1665396,12817412,NaN,Monthly Plan,Monroe St & Blair Blvd,Phase 1 System Area,NaN,44.051473,-123.105138,2019-12-01,...,19603,0072,1.19,00:07:58,keypad,False,0.0,0.0,0.0,0.0
3,1734280,12817414,NaN,UO Monthly Plan,Thompson-Roach Building,Phase 1 System Area,NaN,44.045413,-123.084448,2019-12-01,...,19788,0258,0.43,00:03:13,keypad,False,0.0,0.0,0.0,0.0
4,1702290,12817415,NaN,UO Monthly Plan,Thompson-Roach Building,Phase 1 System Area,NaN,44.045507,-123.084462,2019-12-01,...,19661,0131,0.05,00:06:08,keypad,True,0.0,0.0,0.0,0.0


In [43]:
hub = "Start"

In [46]:
strp = "Eugene"

In [47]:
re = re_df[re_df[hub+' Hub'].str.contains(strp)][[hub+' Latitude', hub+' Longitude']]

In [48]:
re

,Start Latitude,Start Longitude
21,44.037539,-123.083493
32,44.037580,-123.083525
37,44.037539,-123.083493
40,44.037539,-123.083493
61,44.048838,-123.094778
...,...,...
9441,44.037539,-123.083493
9454,44.047807,-123.085165
9467,44.037637,-123.083550
9496,44.047788,-123.085175


In [27]:
hub = "End"

In [29]:
#re = re_df[re_df[hub+'.Hub'].str.isnumeric()][[hub+'.Latitude', hub+'.Longitude']]
re = re_df[re_df[hub+' Hub'].str.isnumeric()][[hub+' Latitude', hub+' Longitude']]
re

,End Latitude,End Longitude
4541,44.040557,-123.117198


In [30]:
re.columns = ['Lat', 'Lon']
re["geom"] = re["Lat"].map(str) + ',' + re['Lon'].map(str)
locator = Nominatim(user_agent="measurements", timeout=10)
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.00001)
tqdm_notebook.pandas()
re['address'] = re['geom'].progress_apply(rgeocode)
re['name'] =re.address.apply(getLoc)

  0%|          | 0/1 [00:00<?, ?it/s]

In [31]:
re.name.values

array(['76'], dtype=object)

In [32]:
re.address.values

array([Location(76, West 18th Avenue, Westmoreland Commercial, Eugene, Lane County, Oregon, 97405, United States, (44.0406689, -123.11734339775421, 0.0))],
      dtype=object)

In [35]:
re.address.values[0].raw

{'place_id': 162223077,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 244974435,
 'lat': '44.0406689',
 'lon': '-123.11734339775421',
 'display_name': '76, West 18th Avenue, Westmoreland Commercial, Eugene, Lane County, Oregon, 97405, United States',
 'address': {'amenity': '76',
  'road': 'West 18th Avenue',
  'commercial': 'Westmoreland Commercial',
  'city': 'Eugene',
  'county': 'Lane County',
  'state': 'Oregon',
  'ISO3166-2-lvl4': 'US-OR',
  'postcode': '97405',
  'country': 'United States',
  'country_code': 'us'},
 'boundingbox': ['44.0406071', '44.040741', '-123.1175184', '-123.1171684']}

In [15]:
re_df.loc[re_df[hub+'.Hub'].str.isnumeric(), hub+' Hub'] = re.name.values

In [16]:
re_df.loc[re_df[hub+'.Hub'].str.isnumeric(), hub+' Hub']

7782                       Dero
7858              Peterson Hall
8165                       Dero
8198                       Dero
8281              Peterson Hall
8329              Peterson Hall
8540                       Dero
8699                       Dero
8748                       Dero
8970    Lillis Business Complex
9126                       Dero
9214              Peterson Hall
Name: Start Hub, dtype: object